In [1]:
import os
import json
import datetime
import pandas as pd
from sqlalchemy import create_engine, text

def load_youtube_mapping_from_db():
    """Load YouTube ID mapping directly from database"""
    try:
        # Create database connection
        engine = create_engine('mysql+pymysql://root@localhost:3306/music_development')
        
        # SQL query to get song title, artist name, and YouTube code
        query = """
        SELECT 
            s.name as song_title,
            CONCAT(a.first_name, ' ', a.last_name) as artist_name,
            s.youtube_code
        FROM songs s
        JOIN artists a ON s.artist_id = a.id
        WHERE s.youtube_code IS NOT NULL AND s.youtube_code != ''
        """
        
        # Execute query and create DataFrame
        df_mapping = pd.read_sql_query(query, engine)
        
        # Rename columns to match expected format
        df_mapping = df_mapping.rename(columns={
            'song_title': 'song_title_db',
            'artist_name': 'artist_name_db',
            'youtube_code': 'youtube_code'
        })
        
        print(f"✅ Loaded {len(df_mapping)} YouTube IDs from database")
        return df_mapping
        
    except Exception as e:
        print(f"❌ Error loading YouTube mapping from database: {e}")
        print(f"💡 Check database connection and credentials")
        return pd.DataFrame(columns=['song_title_db', 'artist_name_db', 'youtube_code'])

def find_youtube_id(song_title, artist, df_mapping):
    """Find YouTube ID using song title and artist - SIMPLIFIED VERSION"""
    if df_mapping.empty:
        return None
    
    # Clean inputs
    clean_artist = str(artist).strip().lower() if artist else ""
    clean_song = str(song_title).strip().lower() if song_title else ""
    
    if not clean_song:
        return None
    
    # Loop through each row in the mapping
    for _, row in df_mapping.iterrows():
        db_song = str(row['song_title_db']).lower().strip() if pd.notna(row['song_title_db']) else ""
        db_artist = str(row['artist_name_db']).lower().strip() if pd.notna(row['artist_name_db']) else ""
        
        # Check exact match
        if db_song == clean_song and db_artist == clean_artist:
            return row['youtube_code']
        
        # Check if song titles match (allowing for variations)
        if db_song == clean_song:
            # Check if artists are similar
            if (clean_artist in db_artist or 
                db_artist in clean_artist or
                clean_artist.replace('the ', '') == db_artist.replace('the ', '')):
                return row['youtube_code']
        
        # Check if one contains the other
        if (clean_song in db_song or db_song in clean_song):
            # Try to match artist
            if clean_artist and db_artist:
                if (clean_artist in db_artist or db_artist in clean_artist):
                    return row['youtube_code']
    
    return None

def scan_mp4_files(folder_path):
    """Scan folder for MP4 files and return sorted list"""
    mp4_files = []
    
    try:
        for file in os.listdir(folder_path):
            if file.lower().endswith('.mp4'):
                mp4_files.append(file)
        
        # Sort alphabetically
        mp4_files.sort()
        
    except Exception as e:
        print(f"Error scanning folder: {e}")
    
    return mp4_files

def generate_songs_with_ids_json(mp4_files, df_mapping, output_file='songs-with-ids.json', additional_folder=True):
    """Generate enhanced JSON file with YouTube IDs"""
    
    enhanced_songs = []
    matched_count = 0
    
    for filename in mp4_files:
        # Remove .mp4 extension
        name_without_ext = filename[:-4]
        
        # Extract song title and artist
        if '_' in name_without_ext:
            parts = name_without_ext.split('_')
            song_title = parts[0].strip()
            artist = ' '.join(parts[1:]).strip()
        else:
            song_title = name_without_ext
            artist = ''
        
        # Find YouTube ID
        youtube_id = find_youtube_id(song_title, artist, df_mapping)
        
        if youtube_id:
            matched_count += 1
        
        song_data = {
            "filename": filename,
            "song_title": song_title,
            "artist": artist,
            "youtube_id": youtube_id if youtube_id else ""
        }
        
        enhanced_songs.append(song_data)
    
    data = {
        "generated_at": datetime.datetime.now().isoformat(),
        "total_songs": len(mp4_files),
        "songs_with_youtube_ids": matched_count,
        "songs_without_youtube_ids": len(mp4_files) - matched_count,
        "songs": enhanced_songs
    }
    
    # List of folders to save to
    save_folders = ['.']  # Current directory
    if additional_folder:
        save_folders.append(additional_folder)
    
    success_count = 0
    for folder in save_folders:
        try:
            output_path = os.path.join(folder, output_file)
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(data, f, indent=2, ensure_ascii=False)
            
            file_size = os.path.getsize(output_path) / 1024  # Size in KB
            print(f"✅ Generated {output_path} ({file_size:.1f} KB)")
            success_count += 1
            
        except Exception as e:
            print(f"❌ Error generating {output_path}: {e}")
    
    if success_count > 0:
        print(f"📊 Total songs: {len(mp4_files)}")
        print(f"📊 Songs with YouTube IDs: {matched_count}")
        print(f"📊 Songs without YouTube IDs: {len(mp4_files) - matched_count}")
        print(f"🕒 Generated at: {data['generated_at']}")
        return True
        
    return False

def analyze_filenames_with_ids(mp4_files, df_mapping):
    """Analyze MP4 filenames and show YouTube ID matches"""
    
    print("\n🎵 Enhanced Filename Analysis with YouTube IDs:")
    
    match_count = 0
    for i, filename in enumerate(mp4_files[:15], 1):  # Show first 15 for preview
        # Remove .mp4 extension
        name_without_ext = filename[:-4]
        
        # Extract song title and artist
        if '_' in name_without_ext:
            parts = name_without_ext.split('_')
            song_title = parts[0].strip()
            artist = ' '.join(parts[1:]).strip()
        else:
            song_title = name_without_ext
            artist = ''
        
        # Find YouTube ID
        youtube_id = find_youtube_id(song_title, artist, df_mapping)
        
        if youtube_id:
            match_count += 1
            status = f"YouTube ID: {youtube_id}"
            match_text = "✅"
        else:
            status = "No YouTube ID found"
            match_text = "❌"
        
        print(f"   {i:2d}. {match_text} '{song_title}' by {artist}")
        if youtube_id:
            print(f"       ↳ {status}")
    
    if len(mp4_files) > 15:
        print(f"   ... and {len(mp4_files) - 15} more songs ...")
    
    # Debug: Show specific song "Nothing's Gonna Stop Us Now"
    print("\n🔍 Debug: Looking for specific song 'Nothing's Gonna Stop Us Now':")
    for filename in mp4_files:
        if "Nothing's Gonna Stop Us Now" in filename:
            name_without_ext = filename[:-4]
            parts = name_without_ext.split('_')
            song_title = parts[0].strip()
            artist = ' '.join(parts[1:]).strip()
            youtube_id = find_youtube_id(song_title, artist, df_mapping)
            print(f"✅ Found: '{song_title}' by {artist}")
            print(f"   YouTube ID match: {youtube_id}")
            break

def generate_upload_checklist(mp4_files, lyrics_folder):
    """Generate a checklist of files to upload to AWS"""
    
    print("\n📋 Upload Checklist for AWS:")
    print("=" * 50)
    
    # Core app files - now includes only songs-with-ids.json
    core_files = ['index.html', 'style.css', 'script.js', 'songs-with-ids.json']
    
    print("\n🚀 Core App Files:")
    for file in core_files:
        if os.path.exists(file):
            file_size = os.path.getsize(file) / 1024  # Size in KB
            print(f"   ✅ {file} ({file_size:.1f} KB)")
        else:
            print(f"   ❌ {file} (MISSING)")
    
    print(f"\n🎵 MP4 Files to Upload ({len(mp4_files)} files):")
    total_size_mb = 0
    for i, mp4_file in enumerate(mp4_files[:5], 1):  # Show first 5 for brevity
        file_path = os.path.join(lyrics_folder, mp4_file)
        if os.path.exists(file_path):
            file_size = os.path.getsize(file_path) / (1024 * 1024)  # Size in MB
            total_size_mb += file_size
            print(f"   ✅ {i:2d}. {mp4_file} ({file_size:.1f} MB)")
        else:
            print(f"   ❌ {i:2d}. {mp4_file} (FILE NOT FOUND)")
    
    if len(mp4_files) > 5:
        print(f"   ... and {len(mp4_files) - 5} more MP4 files ...")
        # Calculate total size of all files
        for mp4_file in mp4_files[5:]:
            file_path = os.path.join(lyrics_folder, mp4_file)
            if os.path.exists(file_path):
                file_size = os.path.getsize(file_path) / (1024 * 1024)
                total_size_mb += file_size
    
    print(f"\n📦 Total estimated upload size: {total_size_mb:.1f} MB")
    print("\n💡 Upload all checked files to your AWS Lyrics folder")

def main():
    """Main function to run the MP4 file scanner with YouTube IDs"""
    
    # Set your Lyrics folder path

    from pathlib import Path
    home = Path.home()                     # expands to C:\Users\<username> on Windows [web:6]
    one_drive = home / "OneDrive"          # or "OneDrive - <YourOrg>"
    LYRICS_FOLDER = one_drive / "A4" / "Data" / "Videos" / "AWS"
    print(LYRICS_FOLDER)

    # LYRICS_FOLDER = r"C:\Users\%USERNAME%\OneDrive\A4\Data\Videos\AWS"
    
    # Additional folder to output JSON files
    # ADDITIONAL_OUTPUT_FOLDER = r"C:\Users\%USERNAME%\OneDrive\A4\Data\videos"
    #ADDITIONAL_OUTPUT_FOLDER = os.chdir(LYRICS_FOLDER.cwd().parent)
    ADDITIONAL_OUTPUT_FOLDER = LYRICS_FOLDER.parent
    print(ADDITIONAL_OUTPUT_FOLDER)
    
    print(f"🎵 MP4 File Scanner with YouTube IDs")
    print(f"📁 Scanning folder: {LYRICS_FOLDER}")
    print("=" * 50)
    
    # Create additional output folder if it doesn't exist
    if not os.path.exists(ADDITIONAL_OUTPUT_FOLDER):
        try:
            os.makedirs(ADDITIONAL_OUTPUT_FOLDER)
            print(f"📁 Created output folder: {ADDITIONAL_OUTPUT_FOLDER}")
        except Exception as e:
            print(f"⚠️  Could not create output folder {ADDITIONAL_OUTPUT_FOLDER}: {e}")
            print("⚠️  JSON files will only be saved to current directory")
            ADDITIONAL_OUTPUT_FOLDER = None
    else:
        print(f"📁 Additional output folder: {ADDITIONAL_OUTPUT_FOLDER}")
    
    # Load YouTube ID mapping from DATABASE
    print("\n📥 Loading YouTube ID mapping from database...")
    df_mapping = load_youtube_mapping_from_db()
    
    # Scan for MP4 files
    mp4_files = scan_mp4_files(LYRICS_FOLDER)
    
    if not mp4_files:
        print("❌ No MP4 files found in the specified folder.")
        return
    
    print(f"\n✅ Found {len(mp4_files)} MP4 files:")
    for i, file in enumerate(mp4_files[:5], 1):  # Show first 5 for brevity
        print(f"   {i:2d}. {file}")
    if len(mp4_files) > 5:
        print(f"   ... and {len(mp4_files) - 5} more files ...")
    
    # Generate enhanced songs-with-ids.json (to both locations)
    generate_songs_with_ids_json(mp4_files, df_mapping, additional_folder=ADDITIONAL_OUTPUT_FOLDER)
    
    # Analyze filenames with YouTube IDs
    analyze_filenames_with_ids(mp4_files, df_mapping)
    
    # Generate upload checklist (updated to only check for songs-with-ids.json)
    generate_upload_checklist(mp4_files, LYRICS_FOLDER)
    
    print("\n" + "=" * 50)
    print("🎉 Script completed successfully!")

if __name__ == "__main__":
    main()

C:\Users\PC1\OneDrive\A4\Data\Videos\AWS
C:\Users\PC1\OneDrive\A4\Data\Videos
🎵 MP4 File Scanner with YouTube IDs
📁 Scanning folder: C:\Users\PC1\OneDrive\A4\Data\Videos\AWS
📁 Additional output folder: C:\Users\PC1\OneDrive\A4\Data\Videos

📥 Loading YouTube ID mapping from database...
✅ Loaded 607 YouTube IDs from database

✅ Found 239 MP4 files:
    1. (Everything I Do) I Do It For You_Bryan Adams.mp4
    2. (I Can't Get No) Satisfaction_Rolling Stones The.mp4
    3. A Change Is Gonna Come_Sam Cooke.mp4
    4. A Day in the Life_Beatles The.mp4
    5. Against the Wind_Bob Seger.mp4
   ... and 234 more files ...
✅ Generated .\songs-with-ids.json (44.0 KB)
✅ Generated C:\Users\PC1\OneDrive\A4\Data\Videos\songs-with-ids.json (44.0 KB)
📊 Total songs: 239
📊 Songs with YouTube IDs: 238
📊 Songs without YouTube IDs: 1
🕒 Generated at: 2026-01-05T16:12:56.453333

🎵 Enhanced Filename Analysis with YouTube IDs:
    1. ✅ '(Everything I Do) I Do It For You' by Bryan Adams
       ↳ YouTube ID: La0IJP